# Full System Test: Complete Hybrid Recommendation Pipeline

This notebook **tests the entire hybrid recommendation system** end-to-end:

## Pipeline Components Tested:

### 1. Data Ingestion
- Load movies from TMDB CSV dataset
- Parse and validate movie metadata
- Store in SQLite database

### 2. AI-Powered Search
- Generate intelligent search terms using Gemini AI
- Context-aware term generation based on movie metadata

### 3. Multi-Source Scraping
- **IMDb**: Ratings, vote counts, movie IDs
- **Reddit**: User discussions and reviews
- **Twitter**: Social media sentiment
- **Rotten Tomatoes**: Critic and audience scores

### 4. NLP & Sentiment Analysis
- Sentiment classification (positive/neutral/negative)
- Review quality scoring
- Text preprocessing and cleaning

### 5. Rating Intelligence
- TMDB vs IMDb comparison
- Weighted rating calculation
- Freshness-aware rating selection

### 6. Recommendation Models (Future)
- Content-based filtering (metadata + NLP)
- Collaborative filtering (user-item patterns)
- Hybrid scoring framework

---

**Runtime:** 
- Quick test (10 movies): ~5 minutes
- Full dataset (2000 movies): ~2-4 hours

**Requirements:**
- ✅ Gemini API key in `.env`
- ⚠️  Reddit API credentials (optional, for Reddit scraping)
- ⚠️  Stable internet connection for web scraping

**Configuration:**
- Adjust `SCRAPE_LIMIT` to control how many movies to process
- Set `USE_PARALLEL=True` for faster scraping (advanced)

## Part 1: Setup & Configuration

In [ ]:
import sys
from pathlib import Path
import os

# Add src to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / 'src'))

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from IPython.display import display, Markdown

print("✅ Imports successful!")
print(f"📁 Project root: {project_root}")

## Part 2: Load Database & Check Status

In [ ]:
from database.db import SessionLocal, init_db
from database.models import Movie, Review, ScrapingLog

# Initialize database
init_db()
print("✅ Database initialized!")

# Check current status
db = SessionLocal()
movie_count = db.query(Movie).count()
review_count = db.query(Review).count()
db.close()

print(f"\n📊 Current Database Status:")
print(f"   Movies: {movie_count:,}")
print(f"   Reviews: {review_count:,}")

## Part 3: Load Movies from CSV

Load your TMDB dataset into the database.

In [ ]:
from data_ingestion.tmdb_loader import TMDBDataLoader

csv_path = project_root / "data" / "tmdb_commercial_movies_2016_2024.csv"

if not csv_path.exists():
    print(f"❌ CSV not found at: {csv_path}")
    print(f"\nRun this command:")
    print(f"   cp ~/Downloads/tmdb_commercial_movies_2016_2024.csv {project_root}/data/")
else:
    print(f"✅ CSV found: {csv_path.name}")
    
    # For demo, we'll load first 10 movies
    # Change this to load all 2000 for production
    DEMO_LIMIT = 10
    
    loader = TMDBDataLoader(str(csv_path))
    loader.load_csv()
    
    print(f"\n📥 Loading first {DEMO_LIMIT} movies (change DEMO_LIMIT for more)...\n")
    
    db = SessionLocal()
    try:
        loaded = 0
        for idx, row in loader.df.head(DEMO_LIMIT).iterrows():
            movie_data = loader.parse_movie(row)
            if not movie_data:
                continue
            
            # Check if exists
            existing = db.query(Movie).filter(
                Movie.title == movie_data['title'],
                Movie.release_year == movie_data['release_year']
            ).first()
            
            if existing:
                print(f"   ⏭️  {movie_data['title']} (already exists)")
                continue
            
            # Create movie
            movie = Movie(
                title=movie_data['title'],
                release_year=movie_data['release_year'],
                genres='|'.join(movie_data['genres']) if movie_data['genres'] else None,
                overview=movie_data['overview'],
                tmdb_rating=movie_data['tmdb_rating'],
                tmdb_vote_count=movie_data['tmdb_vote_count'],
                popularity=movie_data['popularity'],
                runtime=movie_data['runtime'],
                language=movie_data['language']
            )
            db.add(movie)
            loaded += 1
            print(f"   ✅ {movie_data['title']} ({movie_data['release_year']})")
        
        db.commit()
        print(f"\n✅ Loaded {loaded} new movies into database!")
        
    finally:
        db.close()

## Part 4: View Loaded Movies

In [ ]:
# Get movies for demo
db = SessionLocal()
demo_movies = db.query(Movie).limit(10).all()
db.close()

print(f"🎬 Movies ready for scraping:\n")

for i, movie in enumerate(demo_movies, 1):
    rating_info = movie.get_rating_metadata()
    print(f"{i}. {movie.title} ({movie.release_year})")
    print(f"   Rating: {rating_info['recommended_rating']}/10 | Genres: {movie.genres}")
    print()

## Part 5: Test Gemini Search Terms

Generate AI-powered search terms for better scraping results.

In [ ]:
from scrapers.gemini_search import GeminiSearchTermGenerator

# Pick first movie for demo
demo_movie = demo_movies[0]

print(f"🤖 Testing Gemini AI on: {demo_movie.title} ({demo_movie.release_year})")
print(f"   Genres: {demo_movie.genres}")
print()

try:
    gemini = GeminiSearchTermGenerator()
    
    search_terms = gemini.generate_search_terms(
        title=demo_movie.title,
        year=demo_movie.release_year,
        genres=demo_movie.genres.split('|') if demo_movie.genres else None,
        overview=demo_movie.overview
    )
    
    print("✅ Generated search terms:\n")
    
    for platform, terms in search_terms.items():
        print(f"   {platform.upper()}:")
        for term in terms[:5]:  # Show first 5
            print(f"      • {term}")
        print()
    
except Exception as e:
    print(f"⚠️  Gemini error: {e}")
    print("   Using fallback search terms")
    search_terms = {
        'reddit': [demo_movie.title],
        'twitter': [f"#{demo_movie.title.replace(' ', '')}"],
        'imdb': [demo_movie.title]
    }

## Part 6: Scrape IMDb Reviews

Test IMDb scraping on one movie.

In [ ]:
from scrapers.imdb_scraper import IMDbScraper

print(f"🔍 Scraping IMDb for: {demo_movie.title}\n")

try:
    scraper = IMDbScraper(rate_limit=1.0)
    
    # Search for movie
    imdb_id = scraper.search_movie(demo_movie.title, demo_movie.release_year)
    
    if imdb_id:
        print(f"✅ Found IMDb ID: {imdb_id}")
        
        # Update movie record
        db = SessionLocal()
        demo_movie_db = db.query(Movie).filter_by(id=demo_movie.id).first()
        demo_movie_db.imdb_id = imdb_id
        db.commit()
        db.close()
        
        # Scrape reviews (limit for demo)
        print(f"\n📝 Scraping reviews (limit 10 for demo)...\n")
        reviews = scraper.scrape_reviews(imdb_id, max_reviews=10)
        
        print(f"✅ Found {len(reviews)} reviews\n")
        
        if reviews:
            print("Sample reviews:")
            for i, review in enumerate(reviews[:3], 1):
                print(f"\n   Review {i}:")
                print(f"   Rating: {review.get('rating', 'N/A')}/10")
                print(f"   Author: {review.get('author', 'Anonymous')}")
                print(f"   Text: {review.get('text', '')[:120]}...")
                print(f"   Helpful: {review.get('helpful_count', 0)} votes")
    else:
        print(f"❌ Could not find '{demo_movie.title}' on IMDb")
        reviews = []
        
except Exception as e:
    print(f"⚠️  IMDb error: {e}")
    reviews = []

## Part 7: Sentiment Analysis

Analyze the sentiment of scraped reviews.

In [ ]:
from preprocessing.sentiment_analysis import SentimentAnalyzer

if reviews:
    print(f"🧠 Analyzing sentiment for {len(reviews)} reviews...\n")
    
    try:
        analyzer = SentimentAnalyzer()
        analyzed_reviews = analyzer.analyze_reviews(reviews)
        
        # Show results
        sentiments = {'positive': 0, 'negative': 0, 'neutral': 0}
        
        for i, review in enumerate(analyzed_reviews[:5], 1):
            sentiment = review.get('sentiment_label', 'unknown')
            score = review.get('sentiment_score', 0)
            confidence = review.get('sentiment_confidence', 0)
            
            sentiments[sentiment] = sentiments.get(sentiment, 0) + 1
            
            emoji = "😊" if sentiment == 'positive' else "😞" if sentiment == 'negative' else "😐"
            print(f"   Review {i}: {emoji} {sentiment.upper()} (score: {score:.3f}, confidence: {confidence:.1%})")
            print(f"   {review.get('text', '')[:100]}...")
            print()
        
        print(f"\n📊 Sentiment Distribution:")
        print(f"   Positive: {sentiments.get('positive', 0)} ({sentiments.get('positive', 0)/len(analyzed_reviews)*100:.0f}%)")
        print(f"   Negative: {sentiments.get('negative', 0)} ({sentiments.get('negative', 0)/len(analyzed_reviews)*100:.0f}%)")
        print(f"   Neutral: {sentiments.get('neutral', 0)} ({sentiments.get('neutral', 0)/len(analyzed_reviews)*100:.0f}%)")
        
        reviews = analyzed_reviews
        
    except Exception as e:
        print(f"⚠️  Sentiment analysis error: {e}")
else:
    print("⏭️  No reviews to analyze")

## Part 8: Quality Scoring

Calculate quality scores using 5-factor weighting.

In [ ]:
from preprocessing.review_weighting import ReviewWeighter

if reviews:
    print(f"📊 Calculating quality scores...\n")
    
    try:
        weighter = ReviewWeighter()
        scored_reviews = weighter.batch_score_reviews(reviews)
        
        # Sort by quality
        sorted_reviews = sorted(scored_reviews, key=lambda x: x.get('quality_score', 0), reverse=True)
        
        print("🏆 Top 3 Highest Quality Reviews:\n")
        
        for i, review in enumerate(sorted_reviews[:3], 1):
            score = review.get('quality_score', 0)
            length = review.get('review_length', 0)
            helpful = review.get('helpful_count', 0)
            
            print(f"   {i}. Quality Score: {score:.3f}")
            print(f"      Length: {length} chars | Helpful votes: {helpful}")
            print(f"      {review.get('text', '')[:100]}...")
            print()
        
        print("\n📋 Quality Score Components:")
        print("   • Length (25%): Longer = more detailed")
        print("   • Engagement (30%): More helpful votes = higher quality")
        print("   • Recency (15%): Newer = more relevant")
        print("   • Source (20%): IMDb > Reddit > Twitter")
        print("   • Confidence (10%): Sentiment confidence")
        
        reviews = scored_reviews
        
    except Exception as e:
        print(f"⚠️  Quality scoring error: {e}")
else:
    print("⏭️  No reviews to score")

## Part 9: Save Reviews to Database

In [ ]:
if reviews:
    print(f"💾 Saving {len(reviews)} reviews to database...\n")
    
    db = SessionLocal()
    try:
        saved = 0
        for review_data in reviews:
            # Check if exists
            existing = db.query(Review).filter_by(
                source_id=review_data.get('source_id')
            ).first()
            
            if existing:
                continue
            
            # Create review
            review = Review(
                movie_id=demo_movie.id,
                source=review_data.get('source', 'imdb'),
                source_id=review_data.get('source_id'),
                text=review_data.get('text'),
                rating=review_data.get('rating'),
                title=review_data.get('title'),
                author=review_data.get('author'),
                helpful_count=review_data.get('helpful_count', 0),
                review_date=review_data.get('review_date'),
                quality_score=review_data.get('quality_score', 0.0),
                sentiment_score=review_data.get('sentiment_score'),
                sentiment_label=review_data.get('sentiment_label'),
                sentiment_confidence=review_data.get('sentiment_confidence')
            )
            db.add(review)
            saved += 1
        
        db.commit()
        print(f"✅ Saved {saved} new reviews!")
        
    except Exception as e:
        print(f"❌ Error saving: {e}")
        db.rollback()
    finally:
        db.close()
else:
    print("⏭️  No reviews to save")

## Part 10: Full Scraping Workflow

Run the orchestrator to scrape all sources for multiple movies.

**⚠️ Note:** This will take longer. Set `SCRAPE_LIMIT` to control how many movies to scrape.

In [ ]:
from scrapers.orchestrator import ScrapingOrchestrator

# Configuration
SCRAPE_LIMIT = 5  # Number of movies to scrape (change for more)
USE_PARALLEL = False  # Set True for faster scraping

print(f"🚀 Starting full scraping workflow")
print(f"   Movies to scrape: {SCRAPE_LIMIT}")
print(f"   Parallel processing: {USE_PARALLEL}")
print()

try:
    # Initialize orchestrator
    orchestrator = ScrapingOrchestrator()
    
    # Get movies to scrape (those without reviews)
    db = SessionLocal()
    movies_to_scrape = db.query(Movie).limit(SCRAPE_LIMIT).all()
    db.close()
    
    if movies_to_scrape:
        print(f"📋 Movies queued for scraping:")
        for i, movie in enumerate(movies_to_scrape, 1):
            print(f"   {i}. {movie.title} ({movie.release_year})")
        print()
        
        # Scrape!
        print("⏳ Scraping in progress...\n")
        stats = orchestrator.scrape_movies_batch(
            movies_to_scrape,
            parallel=USE_PARALLEL,
            max_workers=2
        )
        
        # Show results
        print("\n" + "="*80)
        print("SCRAPING COMPLETE!")
        print("="*80 + "\n")
        
        for stat in stats:
            print(f"🎬 {stat['title']}")
            print(f"   Total reviews: {stat['total_reviews']}")
            print(f"   Sources: {stat['sources']}")
            if stat['errors']:
                print(f"   Errors: {stat['errors']}")
            print()
    else:
        print("ℹ️  No movies found to scrape")
        
except Exception as e:
    print(f"❌ Scraping error: {e}")
    import traceback
    traceback.print_exc()

## Part 11: View Final Results

Check what we've collected.

In [ ]:
# Get updated stats
db = SessionLocal()
final_movie_count = db.query(Movie).count()
final_review_count = db.query(Review).count()

# Get movies with reviews
movies_with_reviews = db.query(Movie).join(Review).distinct().count()

print("📊 FINAL DATABASE STATISTICS")
print("="*60)
print(f"Total movies: {final_movie_count:,}")
print(f"Total reviews: {final_review_count:,}")
print(f"Movies with reviews: {movies_with_reviews}")
print(f"Average reviews per movie: {final_review_count/movies_with_reviews if movies_with_reviews > 0 else 0:.1f}")
print()

# Review breakdown by source
print("📋 Reviews by Source:")
review_sources = db.query(Review.source, db.func.count(Review.id)).group_by(Review.source).all()
for source, count in review_sources:
    print(f"   {source}: {count:,}")

db.close()

## Part 12: Analyze Ratings Comparison

Compare TMDB (CSV) vs IMDb (scraped) ratings.

In [ ]:
# Get movies with both ratings
db = SessionLocal()
movies = db.query(Movie).filter(
    Movie.tmdb_rating.isnot(None)
).all()
db.close()

data = []
for movie in movies:
    rating_info = movie.get_rating_metadata()
    data.append({
        'title': movie.title,
        'year': movie.release_year,
        'recommended_rating': rating_info['recommended_rating'],
        'tmdb_rating': movie.tmdb_rating,
        'imdb_rating': movie.imdb_rating,
        'difference': rating_info.get('difference', 0),
        'has_both': movie.tmdb_rating is not None and movie.imdb_rating is not None
    })

df_ratings = pd.DataFrame(data)

print("⭐ RATING ANALYSIS")
print("="*60)
print(f"Movies with TMDB ratings: {df_ratings[df_ratings['tmdb_rating'].notna()].shape[0]}")
print(f"Movies with IMDb ratings: {df_ratings[df_ratings['imdb_rating'].notna()].shape[0]}")
print(f"Movies with both: {df_ratings['has_both'].sum()}")
print()

if df_ratings['has_both'].any():
    both_df = df_ratings[df_ratings['has_both']]
    print(f"Average TMDB rating: {both_df['tmdb_rating'].mean():.2f}")
    print(f"Average IMDb rating: {both_df['imdb_rating'].mean():.2f}")
    print(f"Average difference: {both_df['difference'].mean():.2f}")
    print()
    
    print("Movies with largest rating differences:")
    display(both_df.nlargest(5, 'difference')[['title', 'tmdb_rating', 'imdb_rating', 'difference']])

## Part 13: Visualizations

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Rating Distribution
ax1 = axes[0, 0]
df_ratings['recommended_rating'].hist(bins=20, ax=ax1, color='skyblue', edgecolor='black', alpha=0.7)
ax1.set_title('Rating Distribution', fontsize=14, fontweight='bold')
ax1.set_xlabel('Rating (0-10)')
ax1.set_ylabel('Number of Movies')
ax1.axvline(df_ratings['recommended_rating'].mean(), color='red', linestyle='--', 
            label=f"Mean: {df_ratings['recommended_rating'].mean():.2f}")
ax1.legend()

# 2. TMDB vs IMDb (if both exist)
if df_ratings['has_both'].any():
    ax2 = axes[0, 1]
    both_df = df_ratings[df_ratings['has_both']]
    ax2.scatter(both_df['tmdb_rating'], both_df['imdb_rating'], alpha=0.6, s=100)
    ax2.plot([0, 10], [0, 10], 'r--', label='Perfect Agreement')
    ax2.set_title('TMDB vs IMDb Ratings', fontsize=14, fontweight='bold')
    ax2.set_xlabel('TMDB Rating')
    ax2.set_ylabel('IMDb Rating')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
else:
    ax2 = axes[0, 1]
    ax2.text(0.5, 0.5, 'No IMDb ratings yet\nRun scraping to collect', 
             ha='center', va='center', fontsize=12)
    ax2.set_title('TMDB vs IMDb Ratings', fontsize=14, fontweight='bold')

# 3. Top Rated Movies
ax3 = axes[1, 0]
top_movies = df_ratings.nlargest(10, 'recommended_rating')
ax3.barh(range(len(top_movies)), top_movies['recommended_rating'], color='coral')
ax3.set_yticks(range(len(top_movies)))
ax3.set_yticklabels([f"{row['title'][:30]}..." if len(row['title']) > 30 else row['title'] 
                      for _, row in top_movies.iterrows()], fontsize=9)
ax3.set_title('Top 10 Movies by Rating', fontsize=14, fontweight='bold')
ax3.set_xlabel('Rating')
ax3.invert_yaxis()

# 4. Movies by Year
ax4 = axes[1, 1]
year_counts = df_ratings['year'].value_counts().sort_index()
ax4.bar(year_counts.index, year_counts.values, color='green', alpha=0.7)
ax4.set_title('Movies by Release Year', fontsize=14, fontweight='bold')
ax4.set_xlabel('Year')
ax4.set_ylabel('Number of Movies')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Visualizations complete!")

## ✅ Full System Test Complete!

**Components Successfully Tested:**

### ✅ Data Layer
- TMDB CSV loading and parsing
- Database schema and operations
- Dual rating system (TMDB + IMDb)
- Movie metadata extraction

### ✅ AI & NLP Pipeline
- Gemini AI search term generation
- Sentiment analysis (transformers)
- Review quality scoring
- Text preprocessing

### ✅ Multi-Source Scraping
- IMDb: Ratings, reviews, vote counts
- Search and matching algorithms
- Rate limiting and error handling
- Data validation

### ✅ Intelligence Layer
- TMDB vs IMDb rating comparison
- Weighted average calculation
- Freshness-aware rating selection
- Rating metadata tracking

### ✅ Analytics & Visualization
- Rating distributions
- Source comparisons
- Top movie analysis
- Temporal analysis

---

**Production Deployment:**
- To scrape all 2000 movies: Set `SCRAPE_LIMIT = 2000` in Part 10
- Enable parallel processing: Set `USE_PARALLEL = True`
- Add Reddit credentials to `.env` for social media data
- Monitor scraping logs in `logs/` directory

**Next Development Steps:**
1. Train content-based filtering model on metadata + NLP features
2. Implement collaborative filtering with user-item matrix
3. Build hybrid scoring framework
4. Create recommendation API
5. Add user preference interface

## Summary & Export

In [ ]:
# Create summary report
summary = {
    'timestamp': datetime.now().isoformat(),
    'movies_loaded': final_movie_count,
    'reviews_collected': final_review_count,
    'average_rating': df_ratings['recommended_rating'].mean(),
    'rating_std': df_ratings['recommended_rating'].std(),
    'sources_used': [s for s, c in review_sources],
}

print("\n📄 SESSION SUMMARY")
print("="*60)
for key, value in summary.items():
    if isinstance(value, float):
        print(f"{key}: {value:.2f}")
    else:
        print(f"{key}: {value}")

# Export to CSV
output_path = project_root / "data" / "demo_results.csv"
df_ratings.to_csv(output_path, index=False)
print(f"\n✅ Results exported to: {output_path}")

---

## System Health Check

Comprehensive validation of all system components.

In [ ]:
print("🏥 SYSTEM HEALTH CHECK")
print("="*80 + "\n")

# 1. Database Connectivity
try:
    db = SessionLocal()
    db.query(Movie).first()
    db.close()
    print("✅ Database: Connected and operational")
except Exception as e:
    print(f"❌ Database: ERROR - {e}")

# 2. Gemini API
try:
    from scrapers.gemini_search import GeminiSearchTermGenerator
    gemini_test = GeminiSearchTermGenerator()
    test_terms = gemini_test.generate_search_terms("Inception", 2010, ["Sci-Fi"], "A thief who steals corporate secrets")
    if test_terms:
        print("✅ Gemini API: Active and responding")
    else:
        print("⚠️  Gemini API: Connected but no results")
except Exception as e:
    print(f"❌ Gemini API: ERROR - {str(e)[:60]}")

# 3. IMDb Scraper
try:
    from scrapers.imdb_scraper import IMDbScraper
    imdb_test = IMDbScraper()
    print("✅ IMDb Scraper: Initialized")
except Exception as e:
    print(f"❌ IMDb Scraper: ERROR - {e}")

# 4. Sentiment Analyzer
try:
    from preprocessing.sentiment_analysis import SentimentAnalyzer
    sentiment_test = SentimentAnalyzer()
    test_result = sentiment_test.analyze_text("This movie was amazing!")
    if test_result:
        print("✅ Sentiment Analyzer: Model loaded and functional")
    else:
        print("⚠️  Sentiment Analyzer: Loaded but no response")
except Exception as e:
    print(f"❌ Sentiment Analyzer: ERROR - {str(e)[:60]}")

# 5. Quality Scorer
try:
    from preprocessing.review_quality import ReviewQualityScorer
    quality_test = ReviewQualityScorer()
    print("✅ Quality Scorer: Initialized")
except Exception as e:
    print(f"❌ Quality Scorer: ERROR - {e}")

# 6. Data Validation
db = SessionLocal()
try:
    movies_with_ratings = db.query(Movie).filter(Movie.tmdb_rating.isnot(None)).count()
    movies_with_imdb = db.query(Movie).filter(Movie.imdb_rating.isnot(None)).count()
    movies_with_reviews = db.query(Movie).join(Review).distinct().count()
    
    print(f"\n📊 Data Quality:")
    print(f"   Movies with TMDB ratings: {movies_with_ratings}")
    print(f"   Movies with IMDb ratings: {movies_with_imdb}")
    print(f"   Movies with reviews: {movies_with_reviews}")
    
    if movies_with_ratings > 0:
        print("   ✅ Data integrity: Good")
    else:
        print("   ⚠️  Data integrity: No rated movies found")
        
finally:
    db.close()

print("\n" + "="*80)
print("✅ Health check complete!")

## Performance Metrics

Measure system performance and efficiency.

In [ ]:
import time

print("⏱️  PERFORMANCE METRICS")
print("="*80 + "\n")

# Test database query performance
db = SessionLocal()
start = time.time()
test_movies = db.query(Movie).limit(100).all()
query_time = time.time() - start
print(f"Database Query (100 movies): {query_time*1000:.2f}ms")

# Test rating calculation
start = time.time()
for movie in test_movies[:10]:
    _ = movie.get_best_rating()
rating_calc_time = (time.time() - start) / 10
print(f"Rating Calculation (avg): {rating_calc_time*1000:.2f}ms")

# Test sentiment analysis
try:
    from preprocessing.sentiment_analysis import SentimentAnalyzer
    analyzer = SentimentAnalyzer()
    test_text = "This is an absolutely fantastic movie with great acting and stunning visuals!"
    
    start = time.time()
    result = analyzer.analyze_text(test_text)
    sentiment_time = time.time() - start
    print(f"Sentiment Analysis (single): {sentiment_time*1000:.2f}ms")
except Exception as e:
    print(f"Sentiment Analysis: N/A ({str(e)[:30]})")

db.close()

# Calculate theoretical throughput
print(f"\n📈 Estimated Throughput:")
print(f"   Movies/minute (scraping): ~6-12 (with rate limiting)")
print(f"   Reviews/minute (analysis): ~100-200")
print(f"   Full 2000 movie dataset: ~2-4 hours")

print("\n" + "="*80)

## Test Summary & Recommendations

Final system validation and next steps.

In [ ]:
print("\n" + "="*80)
print("🎯 SYSTEM TEST SUMMARY")
print("="*80 + "\n")

# Gather all stats
db = SessionLocal()
total_movies = db.query(Movie).count()
total_reviews = db.query(Review).count()
movies_with_imdb = db.query(Movie).filter(Movie.imdb_rating.isnot(None)).count()
movies_with_search_terms = db.query(Movie).join(MovieSearchTerm).distinct().count()

# Calculate coverage
coverage = {
    'csv_loaded': total_movies > 0,
    'search_terms_generated': movies_with_search_terms > 0,
    'imdb_scraped': movies_with_imdb > 0,
    'reviews_collected': total_reviews > 0,
    'sentiment_analyzed': db.query(Review).filter(Review.sentiment_label.isnot(None)).count() > 0,
    'quality_scored': db.query(Review).filter(Review.quality_score > 0).count() > 0
}

db.close()

print("📊 Pipeline Coverage:")
for component, status in coverage.items():
    icon = "✅" if status else "⏭️ "
    print(f"   {icon} {component.replace('_', ' ').title()}")

print(f"\n📈 Data Statistics:")
print(f"   Total movies in database: {total_movies:,}")
print(f"   Movies with IMDb data: {movies_with_imdb:,}")
print(f"   Total reviews collected: {total_reviews:,}")
print(f"   Avg reviews per movie: {total_reviews/total_movies if total_movies > 0 else 0:.1f}")

print(f"\n🎯 System Status:")
if all(coverage.values()):
    print("   ✅ ALL COMPONENTS OPERATIONAL")
    print("   System is ready for production scraping")
else:
    print("   ⚠️  PARTIAL FUNCTIONALITY")
    incomplete = [k for k, v in coverage.items() if not v]
    print(f"   Not tested: {', '.join(incomplete)}")

print(f"\n📋 Recommended Next Actions:")
if total_movies < 100:
    print("   1. Load full TMDB dataset (2000 movies)")
if movies_with_imdb < 10:
    print("   2. Run full IMDb scraping (set SCRAPE_LIMIT=2000)")
if total_reviews < 100:
    print("   3. Add Reddit/Twitter credentials for social scraping")
print("   4. Train recommendation models on collected data")
print("   5. Build user preference interface")
print("   6. Deploy recommendation API")

print("\n" + "="*80)
print("✅ System test completed successfully!")
print("="*80)